In [1]:
# %reload_ext autoreload
# %autoreload 2

# import os
# import sys

# # Add the root folder to the module search path
# # Get the current directory
# current_directory = os.getcwd()

# # Move two levels up (go to the parent directory of the parent directory)
# two_levels_up_directory = os.path.dirname(os.path.dirname(current_directory))

# print(two_levels_up_directory)

# sys.path.append(two_levels_up_directory)

In [2]:
# !pip install pyngrok
# !pip install mlflow
# !pip install boto3

### Accelerate Config (Once per machine)
reference: https://huggingface.co/docs/accelerate/basic_tutorials/notebook


```
(pykoi) $ accelerate config
----------------------------------------------------------------------------------In which compute environment are you running?
This machine                                                                      
----------------------------------------------------------------------------------Which type of machine are you using?                                              
multi-CPU                                                                         
How many different machines will you use (use more than 1 for multi-node training)? [1]: 1                                                                          
Do you want to use Intel PyTorch Extension (IPEX) to speed up training on CPU? [yes/NO]:No                                                                          
Do you wish to optimize your script with torch dynamo?[yes/NO]:No                 
How many CPU(s) should be used for distributed training? [1]:8                    
----------------------------------------------------------------------------------Do you wish to use FP16 or BF16 (mixed precision)?
fp16                                                                              
accelerate configuration saved at /home/ubuntu/.cache/huggingface/accelerate/default_config.yaml   
```

In [3]:
from pykoi.rlhf import RLHFConfig

from accelerate import notebook_launcher

import mlflow
import datetime


/home/ubuntu/miniconda3/envs/koi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import json
import os
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional

import evaluate
import numpy as np
import torch
from pykoi.chat.db.qa_database import QuestionAnswerDatabase
from accelerate import Accelerator, notebook_launcher
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftConfig, PeftModel, TaskType, get_peft_model

from tqdm import tqdm
from transformers import (Adafactor, AutoModelForCausalLM,
                          AutoModelForSequenceClassification, AutoTokenizer,
                          Trainer, TrainerCallback, TrainingArguments, logging,
                          pipeline, set_seed)
from transformers.utils import PushToHubMixin
from trl import (AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer,
                 SFTTrainer)
from trl.core import LengthSampler
from trl.trainer.utils import ConstantLengthDataset, PeftSavingCallback

In [5]:
QA_CSV_HEADER_ID = 'ID'
QA_CSV_HEADER_QUESTION = 'Question'
QA_CSV_HEADER_ANSWER = 'Answer'
QA_CSV_HEADER_VOTE_STATUS = 'Vote Status'
QA_CSV_HEADER_TIMESTAMPS = 'Timestamp'
QA_CSV_HEADER = (
    QA_CSV_HEADER_ID,
    QA_CSV_HEADER_QUESTION,
    QA_CSV_HEADER_ANSWER,
    QA_CSV_HEADER_VOTE_STATUS,
    QA_CSV_HEADER_TIMESTAMPS
)

In [6]:
class RL(Trainer):
    def __init__(self, rlhf_config: RLHFConfig):
        self._rlhf_config = rlhf_config
        self.accelerator = Accelerator()
        self.num_proc = self._rlhf_config.num_workers if not self._rlhf_config.streaming else None
        set_seed(rlhf_config.seed) ## TODO: how to set seed properly in __init__?

        self.ppo_config=PPOConfig(
            steps=self._rlhf_config.total_ppo_epochs,
            model_name=self._rlhf_config.base_model_path,
            learning_rate=self._rlhf_config.learning_rate,
            batch_size=self._rlhf_config.ppo_batch_size,
            mini_batch_size=self._rlhf_config.mini_batch_size,
            gradient_accumulation_steps=self._rlhf_config.gradient_accumulation_steps,
            optimize_cuda_cache=True,
            early_stopping=self._rlhf_config.early_stopping,
            target_kl=self._rlhf_config.target_kl,
            ppo_epochs=self._rlhf_config.ppo_epochs,
            seed=self._rlhf_config.seed,
            init_kl_coef=self._rlhf_config.init_kl_coef,
            adap_kl_ctrl=self._rlhf_config.adap_kl_ctrl,
            # accelerator_kwargs=self._rlhf_config.accelerator_kwargs,
            )
        
        ## Load the base model and tokenizer and define the PPO Trainer for RL
        self.base_tokenizer = self.create_tokenizer(rlhf_config.base_model_path)
        self.base_dataset=self.create_dataset(self.base_tokenizer)
        self.base_model = AutoModelForCausalLMWithValueHead.from_pretrained(
            rlhf_config.base_model_path,
            load_in_8bit=rlhf_config.load_in_8bit,
            # is_loaded_in_8bit = True, # TODO TypeError: LlamaForCausalLM.__init__() got an unexpected keyword argument 'is_loaded_in_8bit'
            # torch_dtype=torch.float16, 
            device_map={"": Accelerator().local_process_index},
            peft_config=rlhf_config.lora_config_rl, 
        )
        self.ppo_trainer = PPOTrainer(
            config=self.ppo_config,
            model=self.base_model,
            ref_model=None,
            tokenizer=self.base_tokenizer,
            dataset=self.base_dataset,
            data_collator=self.data_collator,
            # optimizer=optimizer,
            # peft_config=lora_config, ## PPOTrainer doesn't support parameter peft_config
        )
        self.base_kwargs = {
            # "min_length": -1,
            "top_k": rlhf_config.top_k,
            "top_p": rlhf_config.top_p,
            "do_sample": rlhf_config.do_sample,
            "pad_token_id": self.base_tokenizer.pad_token_id,
            "eos_token_id": rlhf_config.eos_token_id,
            "max_length": rlhf_config.output_max_length
        }

        ## Load the reward model and tokenizer and define the reward pipeline
        self.reward_tokenizer = self.create_tokenizer(rlhf_config.reward_model_path)
        self.reward_dataset=self.create_dataset(self.reward_tokenizer)
        self.reward_model = AutoModelForSequenceClassification.from_pretrained(
            rlhf_config.reward_model_path, 
            num_labels=1,
            # torch_dtype=torch.bfloat16,
            load_in_8bit=True,
            device_map={"": Accelerator().local_process_index}
        )
        self.reward_kwargs = {
            "return_all_scores": True,
            "function_to_apply": "none",
            "batch_size": self._rlhf_config.ppo_batch_size,
            "truncation": True,
            "max_length": self._rlhf_config.output_max_length
        }
        self.reward_pipe = pipeline(
            "sentiment-analysis",
            model=self.reward_model,
            # device_map={"": Accelerator().local_process_index},
            # model_kwargs={"load_in_8bit": True},
            model_kwargs=self.reward_kwargs,
            tokenizer=self.reward_tokenizer,
            return_token_type_ids=False,
        )

        
    def create_tokenizer(self, model_name):
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        if getattr(tokenizer, "pad_token", None) is None:
            tokenizer.pad_token = tokenizer.eos_token
        return tokenizer


    def data_collator(self, data):
        return dict((key, [d[key] for d in data]) for key in data[0])


    def create_dataset(self, tokenizer):
        """
        Build dataset for training. This builds the dataset from `load_dataset`, one should
        customize this function to train the model on its own dataset.
        """
        args = self._rlhf_config
        if args.dataset_type == "local_db":
            qa_database = QuestionAnswerDatabase()
            my_data_pd = qa_database.retrieve_all_question_answers_as_pandas()
            my_data_pd = my_data_pd[my_data_pd[QA_CSV_HEADER_VOTE_STATUS]=="up"]
            my_data_pd = my_data_pd[[QA_CSV_HEADER_ID,
                                     QA_CSV_HEADER_QUESTION,
                                     QA_CSV_HEADER_ANSWER]]
            print("My local database has {} samples".format(my_data_pd.shape[0]))
            dataset = Dataset.from_dict(my_data_pd)
        elif args.dataset_type == "local_csv":
            dataset = load_dataset('csv', data_files=args.dataset_name)
            dataset = dataset[args.split] # Convert DatasetDict to Dataset
        elif args.dataset_type == "huggingface":
            dataset = load_dataset(
                args.dataset_name,
                data_dir=args.dataset_subset_sft,
                split=args.split,
                use_auth_token=True,
                num_proc=self.num_proc,
                streaming=args.streaming,
            )
            dataset = dataset[args.split] # Convert DatasetDict to Dataset
        else:
            raise FileNotFoundError(f"No (supported) data files or dataset script found {args.dataset_type}")
        
        # dataset = dataset.train_test_split(test_size=args.train_test_split_ratio, 
        #                                    seed=args.seed)
        # print(f"Size of the train set: {len(dataset['train'])}. \
        #       Size of the validation set: {len(dataset['test'])}")
        
        # dataset = dataset.select(range(self._rlhf_config.dataset_subset_rl_train))

        def preprocess_function(examples):
            queries = ["Question: " + q + "\n\nAnswer: " for q in examples[QA_CSV_HEADER_QUESTION]]
            input_ids = [tokenizer(q, truncation=True)["input_ids"] for q in queries]
            return {"query": queries, "input_ids": input_ids}

        dataset = dataset.map(
            preprocess_function,
            batched=True,
            num_proc=24, ## TODO self.num_proc,
            remove_columns=dataset.column_names,
        )
        dataset = dataset.filter(lambda x: len(x["input_ids"]) < self._rlhf_config.max_seq_length, 
                       batched=False)
        dataset.set_format(type="torch") ## TODO

        return dataset


    def _train(self):
        for epoch, batch in tqdm(enumerate(self.ppo_trainer.dataloader)):
            if epoch >= self._rlhf_config.total_ppo_epochs:
                break
            ## embed the questions and responses to tensors
            question_tensors = batch["input_ids"]
            response_tensors = self.ppo_trainer.generate(
                question_tensors,
                return_prompt=False,
                length_sampler=LengthSampler(self._rlhf_config.output_min_length, 
                                             self._rlhf_config.output_max_length),
                **self.base_kwargs,
            )
            batch[QA_CSV_HEADER_ANSWER] = self.base_tokenizer.batch_decode(response_tensors, 
                                                                 skip_special_tokens=True)
            # compute rewards and run PPO
            texts = [q + r for q, r in zip(batch["query"], batch[QA_CSV_HEADER_ANSWER])]
            pipe_outputs = self.reward_pipe(texts, **self.reward_kwargs)
            rewards = [torch.tensor(output[0]["score"] - self._rlhf_config.reward_baseline) \
                       for output in pipe_outputs]
            stats = self.ppo_trainer.step(question_tensors, response_tensors, rewards)
            self.ppo_trainer.log_stats(stats, batch, rewards)

            ## save weights
            if self._rlhf_config.save_freq and epoch and \
                epoch % self._rlhf_config.save_freq == 0:
                self.ppo_trainer.save_pretrained(
                    os.path.join(self._rlhf_config.output_dir, f"rlhf_rl_step_{epoch}"))
                
    def train(self, num_processes=1):
        notebook_launcher(self._train)



In [ ]:
# Set up mlflow experiment name.

# mlflow.set_tracking_uri("http://x.x.x.x:5000")
experiment = "rlhf_step3_rl"
current_time = str(datetime.datetime.now())
mlflow_experiment_name = '/'.join([experiment, current_time])

try:
    mlflow.end_run()
except:
    print("No mlflow run in progress")

mlflow.set_experiment(mlflow_experiment_name)

In [8]:
# Set pykoi parameters.
base_model_path = "elinas/llama-7b-hf-transformers-4.29"
dataset_type = "local_db"
reward_model_path = "models/rlhf_step2_rw"
save_freq = 100
ppo_batch_size = 32
ppo_epochs = 4
total_epochs = 5
total_ppo_epochs = 5
trained_model_path = "./models/rlhf_step3_rl"

In [9]:
# Manually log pykoi parameters into mlflow. Torch level parameters are automatically logged.
mlflow.log_param("pykoi_base_model_path", base_model_path)
mlflow.log_param("pykoi_dataset_type", dataset_type)
mlflow.log_param("pykoi_reward_model_path", reward_model_path)
mlflow.log_param("pykoi_save_freq", save_freq)
mlflow.log_param("pykoi_ppo_batch_size", ppo_batch_size)
mlflow.log_param("pykoi_ppo_epochs", ppo_epochs)
mlflow.log_param("pykoi_total_epochs", total_epochs)
mlflow.log_param("pykoi_total_ppo_epochs", total_ppo_epochs)
mlflow.log_param("pykoi_trained_model_path", trained_model_path)

'./models/rlhf_step3_rl'

In [12]:
# Training metrics are automatically logged into mlflow.
from accelerate import notebook_launcher

config = RLHFConfig(base_model_path=base_model_path,
                          dataset_type=dataset_type,
                          reward_model_path=reward_model_path,
                          save_freq=save_freq,
                          ppo_batch_size=ppo_batch_size,
                          ppo_epochs=ppo_epochs,
                          total_epochs=total_epochs,
                          total_ppo_epochs=total_ppo_epochs
                          )
rlhf_step3_rl = RL(config)
rlhf_step3_rl.train(trained_model_path, num_processes=1)

TypeError: RLHFConfig.__init__() got an unexpected keyword argument 'total_ppo_epochs'

In [ ]:
# Save the trained model into mlflow artifacts.
mlflow.log_artifacts(trained_model_path)

In the terminal, run
```
mlflow ui
```
and go to http://127.0.0.1:5000 in the browser to view the experiment in the UI.